<a href="https://colab.research.google.com/github/Vishy-A/DeepLearning/blob/main/4106HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
from google.colab import drive
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
from collections import OrderedDict
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader


In [101]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.4915, 0.4823, 0.4468),
                           (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified
Files already downloaded and verified


In [102]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [103]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA (GPU) is available.")
else:
    device = torch.device("cpu")
    print("CUDA (GPU) is not available. Using CPU.")

CUDA (GPU) is available.


In [104]:
def trainingloop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('Epoch {}, Training loss {}'.format(epoch, loss_train / len(train_loader)))

In [105]:
def validateloop(model, train_loader, val_loader):
      for name, loader in [("train", train_loader), ("val", val_loader)]:
          correct = 0
          total = 0



          alllabels = []
          allpreds = []

          with torch.no_grad():
              for imgs, labels in loader:
                  outputs = model(imgs)
                  _, predicted = torch.max(outputs, dim=1)
                  total += labels.shape[0]
                  correct += int((predicted == labels).sum())

                  alllabels.extend(labels.cpu().numpy())
                  allpreds.extend(predicted.cpu().numpy())

          print("Accuracy {}: {:.2f}".format(name , correct / total))

          alllabels = np.array(alllabels)
          allpreds = np.array(allpreds)

          report = classification_report(alllabels, allpreds)
          print(f"Classification Report {name}:\n{report}")



In [106]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return torch.softmax(x, dim=1)

In [107]:
trainloader = DataLoader(cifar10, batch_size=64, shuffle=True)
valloader = DataLoader(cifar10_val, batch_size=64, shuffle=False)

In [108]:
model = Network()
lossfunc = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
epochsReported = [1, 10, 20]

In [109]:
trainingloop(
    n_epochs = 20,
    optimizer = optimizer,
    model = model,
    loss_fn = lossfunc,
    train_loader = valloader,
)

Epoch 1, Training loss 2.3022934813408336
Epoch 10, Training loss 2.2760231434159977
Epoch 20, Training loss 2.2084268415050143


In [110]:
validateloop(model, trainloader, valloader)

Accuracy train: 0.25
Classification Report train:
              precision    recall  f1-score   support

           0       0.29      0.47      0.36      5000
           1       0.00      0.00      0.00      5000
           2       0.00      0.00      0.00      5000
           3       0.00      0.00      0.00      5000
           4       0.00      0.00      0.00      5000
           5       0.25      0.26      0.25      5000
           6       0.20      0.59      0.30      5000
           7       0.00      0.00      0.00      5000
           8       0.34      0.58      0.43      5000
           9       0.24      0.63      0.34      5000

    accuracy                           0.25     50000
   macro avg       0.13      0.25      0.17     50000
weighted avg       0.13      0.25      0.17     50000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy val: 0.26
Classification Report val:
              precision    recall  f1-score   support

           0       0.31      0.52      0.38      1000
           1       0.00      0.00      0.00      1000
           2       0.00      0.00      0.00      1000
           3       0.00      0.00      0.00      1000
           4       0.00      0.00      0.00      1000
           5       0.26      0.26      0.26      1000
           6       0.20      0.58      0.30      1000
           7       0.00      0.00      0.00      1000
           8       0.34      0.60      0.44      1000
           9       0.23      0.62      0.34      1000

    accuracy                           0.26     10000
   macro avg       0.13      0.26      0.17     10000
weighted avg       0.13      0.26      0.17     10000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
